In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style()

# a

Мысль номер раз: сделать почти как на практике, следующую штуку:

Представим, что мы делаем фотографии попарно, сначала 1 и 5, потом 2 и 6, и так далее, при этом каждый раз Маша говорит нам, какая из этих фотографий лучше. Тогда выразим математически высказывание "качество фотографии на телефон А ниже качества фотографии на телефон Б" как:

$X$ - случайная величина, равная качеству фотографии на телефон А, $Y$ - с.в., равная качеству фотографии на телефон Б, тогда $P(X < Y) > .5$. Соответственно, это нашей нулевой гипотезой будет то, что $P(X < Y) = .5$, а альтернативаной - что $P(X < Y) > .5$.

Так как все четыре эксперимента, которые мы провели до этого, были независимыми и показали, что $X<Y$, то при $P(X < Y) = .5$ вероятность этого будет меньше или равна $0.0625$, что больше чем уровень значимости, то есть гипотеза не отвергается.

Однако можно подумать о том, что наша интерпретация эксперимента никак не учитывает тот факт, что ВСЕ измерения одного фотоаппарата были лучше ВСЕХ измерений другого. Как насчёт изменить наш взгляд на эту ситуацию и рассмотреть все попарные сравнения?

Тогда вероятность будет равна $2^{-16}$, а это очень маленькая величина, то есть нулевая гипотеза будет отвергнута. Но тут можно обратить внимание на тот факт, что на самом деле это сильно зависит от распределения $X$ и $Y$, в отличии от первого теста. Давайте промоделируем обе ситуации:

In [2]:
# 1
def check_1(x: np.ndarray, y: np.ndarray) -> bool:
    for i in range(len(x)):
        if x[i] >= y[i]:
            return False
    return True

# We do not depend on distribution
successes = 0
total = 1_000_000

for _ in range(total):
    x = np.random.uniform(size=4)
    y = np.random.uniform(size=4)
    if check_1(x, y):
        successes += 1
        
successes / total

0.062507

In [3]:
def check_2(x: np.ndarray, y: np.ndarray) -> bool:
    for X in x:
        for Y in y:
            if X >= Y:
                return False
    return True

# We DO depend on distribution

def test_with_gen(gen, total: int = 1_000_000, size: int = 4) -> float:
    successes = 0
    
    for _ in range(total):
        x = gen(size=size)
        y = gen(size=size)
        if check_2(x, y):
            successes += 1

    return successes / total

# So lets test with uniform and normal
test_with_gen(np.random.uniform), test_with_gen(np.random.normal)

(0.014417, 0.014238)

Это показывает, что я облажался в своих высказываниях! Потому что на самом деле второй тест можно свести к тому, что *минимум* из выборки $y$ больше, чем *максимум* из выборки $x$. Это считается иначе:

$$P(Y_{min} > X_{max}) = \sum P(Y_i > X_j | (Y_i \leq Y_{k}) \text{ and } (X_j \geq X_k)) \cdot P(Y_i \leq Y_k) \cdot (X_j \geq X_k)) = 16 \cdot (2^{-1} \cdot 2^{-3} \cdot 2^{-3})$$

Всё ещё не сходится, давайте поэкспериментируем с количеством тестов:

In [4]:
results = [test_with_gen(np.random.uniform, size=sz, total=10_000_000) for sz in range(1, 7)]

In [5]:
results

[0.4999309, 0.1666464, 0.0500196, 0.0141981, 0.0039812, 0.0010797]

Выглядит забавно, потому что я не могу узнать эту последовательность. А что если посмотреть на обратную?

In [6]:
1 / np.array(results)

array([  2.00027644,   6.00072969,  19.99216307,  70.43195921,
       251.18054858, 926.18319904])

Заходим в OEIS, находим [вот это](https://oeis.org/A000984). Стыдно, блин, так облажаться и забыть про это. В общем, да, второй тест даёт нам достаточную значимость отказа.

# b

Здесь сразу сделаем второй тест из вышеописанных, собственно, посчитав количество вариантов для последовательности оказаться ровно в таком порядке.

In [7]:
def check_3(x: np.ndarray, y: np.ndarray) -> bool:
    x.sort()
    y.sort()
    return x[2] < y[0] and x[3] > y[0] and x[3] < y[1]

successes = 0
    
for _ in range(total):
    x = np.random.normal(size=4)
    y = np.random.normal(size=4)
    if check_3(x, y) or check_2(x, y):
        successes += 1

successes / total

0.028696

Теперь значимость будет равна $\frac{2}{70}$, что всё ещё более значимо, чем $0.05$.

## P.S.

Второй тест на самом деле честно работает только для одинаково распределённых величин, если мы засунем туда разные величины с одинаковым "центром" (ну типа как матожидание только вероятность быть справа и слева равна), то все сломается.

# c

In [8]:
tws1, tws2 = pd.read_csv('data/01/tws1.csv', sep=' ').to_numpy().flatten(), pd.read_csv('data/01/tws2.csv', sep=' ').to_numpy().flatten()

In [9]:
from scipy.stats import mannwhitneyu as mnu

mnu(tws1, tws2)

MannwhitneyuResult(statistic=9456.0, pvalue=0.7733823497835443)

MNU говорит нет.

# d

## a

Получаем сумму рангов для $X$ равную $1+2+3+4$, сумму рангов для $Y$, равную $5+6+7+8$. Тогда:

$$
\begin{split}
    &U_1 = 4 \cdot 4 + \frac{4 \cdot 5}{2} - R_1 = 16 + 10 - 10 = 16\\
    &U_2 = 4 \cdot 4 + \frac{4 \cdot 5}{2} - R_2 = 16 + 10 - 26 = 0
\end{split}
$$

Согласно [таблице](https://math.usask.ca/~laverty/S245/Tables/wmw.pdf), получаем **достаточный** уровень значимости чтобы отклонить нулеву гипотезу.

## b

Получаем сумму рангов для $X$ равную $1+2+3+5$, сумму рангов для $Y$, равную $4+6+7+8$. Тогда:

$$
\begin{split}
    &U_1 = 4 \cdot 4 + \frac{4 \cdot 5}{2} - R_1 = 16 + 10 - 11 = 15\\
    &U_2 = 4 \cdot 4 + \frac{4 \cdot 5}{2} - R_2 = 16 + 10 - 25 = 1
\end{split}
$$

Согласно [таблице](https://math.usask.ca/~laverty/S245/Tables/wmw.pdf), получаем **недостаточный** уровень значимости чтобы отклонить нулеву гипотезу.